In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import glob
import keras.utils
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation, Permute, GRU, Reshape
from keras.layers import Conv1D, MaxPooling2D, Flatten, Conv2D, BatchNormalization, Lambda, Bidirectional, concatenate
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import RMSprop
import graphviz
import keras.backend as K
K.set_image_data_format('channels_last')
from tensorflow import set_random_seed
set_random_seed(77)
np.random.seed(17)

In [ ]:
spec_path = '../input/spect-scaled/all_spects_scaled.npz'
with open(spec_path, 'rb') as handle:
    all_spects = np.load(handle)['arr_0']
    
info_path = '../input/track-info/track_info.pickle'
with open(info_path, 'rb') as handle:
    track_info = pickle.load(handle)

In [ ]:
track_ids = track_info.index
track_id_dict = dict(zip(track_ids,all_spects))

In [ ]:
def make_dataset_splits(split):
    mask = track_info.split == split
    ids = track_info.loc[mask].index
    data = np.asarray([track_id_dict[i] for i in ids])
    
    return data

In [ ]:
def make_label_splits(split):
    mask = track_info.split == split
    labels = track_info.loc[mask, 'labels']
    labels = np.asarray([np.array(l) for l in labels])
    return labels

In [ ]:
X_train = make_dataset_splits('training')
X_val = make_dataset_splits('validation')
X_test = make_dataset_splits('test')

In [ ]:
enc = LabelEncoder()
track_info['labels'] = enc.fit_transform(track_info['genre_top'])

y_cat = keras.utils.to_categorical(track_info['labels'].values)
track_info['labels'] = [y.tolist() for y in y_cat]

y_train = make_label_splits('training')
y_val = make_label_splits('validation')
y_test = make_label_splits('test')

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, model_name,  epochs = 20, batch_size = 64):
    
    best_model_path = './models/'+ model_name + '/weights.best.h5'
    best_model = ModelCheckpoint(best_model_path, monitor='val_acc',
                                          save_best_only=True, mode='max')
    rp_callback = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_delta=0.0)
    
    tensorboard_path = './models/'+ model_name + '/logs'
    tensorboard = TensorBoard(log_dir = tensorboard_path)
    
    hist = model.fit(X_train, y_train, epochs = epochs,batch_size = batch_size, validation_data = (X_val,y_val),
                    callbacks = [best_model,rp_callback, tensorboard])
    return model, hist

In [ ]:
def cnn(param_grid):
    
    n_filters = param_grid['n_filters']
    filter_size = param_grid['filter_size']
    pool_size = param_grid['pool_size']
    dropout_rate = param_grid['dropout_rate']
    n_cov_layers = param_grid['n_cov_layers']
    n_dense_nodes = param_grid['n_dense_nodes']
    input_shape = param_grid['input_shape']
    
    model = Sequential()
    for i in range(n_cov_layers):
        model.add(Conv2D(filters = n_filters, kernel_size = filter_size, activation = 'relu', 
                         input_shape = (input_shape[-3],input_shape[-2],1), padding = 'same',
                        data_format = 'channels_last', name = 'conv2d_' + str(i)))
        model.add(BatchNormalization( name = 'batch_norm_' + str(i)))
        model.add(Activation('relu', name = 'relu_' + str(i)))
        model.add(MaxPooling2D(pool_size = pool_size, data_format = "channels_last", name = 'max_pool2d_' + str(i)))
        model.add(Dropout(dropout_rate, name = 'dropout_' + str(i)))
    
    model.add(Flatten(name = 'flatten'))
    model.add(Dense(n_dense_nodes , activation = 'relu', name = 'dense_1'))
    model.add(Dense(n_dense_nodes , activation = 'relu', name='dense_2'))
    model.add(Dense(8, activation = 'softmax', name = 'output'))
    
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
param_grid = {'n_filters': 60,
             'filter_size': (5,5),
             'pool_size': (2,2),
              'input_shape': X_train.shape,
             'dropout_rate': 0.3,
             'n_cov_layers': 5,
             'n_dense_nodes':240}

cnn1 = cnn(param_grid)
cnn1_fitted , cnn_history = train_model(cnn1, X_train, y_train, X_val, y_val, model_name  = 'cnn1')

In [ ]:
train_loss_cnn, train_acc_cnn = cnn1.evaluate(X_train, y_train )
print('Training set Accuracy: ',train_acc_cnn)

In [ ]:
val_loss_cnn, val_acc_cnn = cnn1.evaluate(X_val, y_val)
print('Validation set Accuracy: ',val_acc_cnn)

In [ ]:
test_loss_cnn, test_acc_cnn = cnn1.evaluate(X_test, y_test)
print('Test set Accuracy: ',test_acc_cnn)

In [ ]:
def crnn(param_grid):
    
    n_filters = param_grid['n_filters']
    filter_size = param_grid['filter_size']
    pool_size = param_grid['pool_size']
    dropout_rate = param_grid['dropout_rate']
    n_cov_layers = param_grid['n_cov_layers']
    n_recurrent_layers = param_grid['n_recurrent_layers']
    n_time_layers = param_grid['n_time_layers']
    n_dense_nodes = param_grid['n_dense_nodes']
    n_time_dst_nodes = param_grid['n_time_dst_nodes']
    l2 = param_grid['l2']
    output_shape = param_grid['output_shape']
    input_shape = param_grid['input_shape']
    
    
    input_layer = Input(shape=(input_shape[-3], input_shape[-2], input_shape[-1]),
                       name = 'input')
    layer = input_layer
    for i in range(n_cov_layers):
        layer = Conv2D(filters = n_filters, kernel_size=filter_size, padding='same', 
                       name = 'conv2d_' + str(i))(layer)
        layer = BatchNormalization(axis=3, name = 'batch_norm' + str(i))(layer)
        layer = Activation('relu', name = 'relu' + str(i))(layer)
        layer = MaxPooling2D(pool_size=pool_size, name = 'pool2d' + str(i))(layer)
    
    layer = Permute((2, 1, 3), name = 'permute')(layer)
    layer = Reshape((input_shape[-2], -1), name = 'reshape')(layer)
    layer = LSTM(96, return_sequences=False, name = 'lstm')(layer)
    
    layer = Dropout(dropout_rate, name = 'dropout_lstm')(layer)
    layer = Dense(n_dense_nodes, kernel_regularizer=regularizers.l2(l2), name = 'dense_regularizer')(layer)
    

    output_layer = Dense(8, activation = 'softmax', name = 'output')(layer)
    
    model = Model(inputs = input_layer, outputs = output_layer)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics = ['accuracy'])

    return model

In [ ]:
crnn_param_grid = {'n_filters': 56,
                   'filter_size': (2,2),
                   'pool_size': (2,2),
                   'dropout_rate': 0.2,
                   'n_cov_layers': 5, 
                   'n_recurrent_layers':2,
                   'n_time_layers': 1,
                   'n_time_dst_nodes': 128,
                   'n_dense_nodes':240,
                   'l2': .001,
                   'output_shape': y_train.shape }

crnn1 = crnn(X_train, crnn_param_grid)
crnn1_fitted , crnn_history = train_model(crnn1, X_train, y_train, X_val, y_val, model_name = 'crnn', epochs = 60)

In [ ]:
train_loss_crnn, train_acc_crnn = crnn1.evaluate(X_train, y_train)
print('Training set Accuracy: ',train_acc_crnn)

In [ ]:
val_loss_crnn, val_acc_crnn = crnn1.evaluate(X_val, y_val)
print('Validation set Accuracy: ',val_acc_crnn)

In [ ]:
test_loss_crnn, test_acc_crnn = crnn1.evaluate(X_test, y_test)
print('Test set Accuracy: ',test_acc_crnn)

In [ ]:
def crnn_parallel(input_data, param_grid):
    
    n_filters = param_grid['n_filters']
    filter_size = param_grid['filter_size']
    pool_size = param_grid['pool_size']
    dropout_rate = param_grid['dropout_rate']
    n_cov_layers = param_grid['n_cov_layers']
    n_recurrent_layers = param_grid['n_recurrent_layers']
    n_time_layers = param_grid['n_time_layers']
    n_dense_nodes = param_grid['n_dense_nodes']
    n_time_dst_nodes = param_grid['n_time_dst_nodes']
    l2 = param_grid['l2']
    output_shape = param_grid['output_shape']
    
    
    input_layer = Input(shape=(input_data.shape[-3], input_data.shape[-2], input_data.shape[-1]),
                       name = 'input')
    layer = input_layer
    for i in range(n_cov_layers):
        layer = Conv2D(filters = n_filters[i], kernel_size = filter_size, padding = 'valid', 
                       strides = 1, name = 'conv2d_' + str(i))(layer)
        layer = BatchNormalization(axis = 3, name = 'batch_norm' + str(i))(layer)
        layer = Activation('relu', name = 'relu' + str(i))(layer)
        layer = MaxPooling2D(pool_size = pool_size[i], name = 'pool2d' + str(i))(layer)
        layer = Dropout(dropout_rate, name = 'dropout' + str(i))(layer)
    
    layer = Flatten()(layer)
    
    r_layer = MaxPooling2D(pool_size[5], name = 'pool_lstm')(input_layer)
    r_layer = Lambda(lambda x: K.squeeze(x, axis= -1))(r_layer)
    
    r_layer = Bidirectional(GRU(64), merge_mode='concat')(r_layer)

    concat = concatenate([layer, r_layer], axis=-1, name ='concat')
    
    output_layer = Dense(8, activation = 'softmax', name = 'output')(concat)
 
    opt = RMSprop(lr=0.0005)
    model = Model(inputs = input_layer, outputs = output_layer)
    model.compile(optimizer=opt, loss='categorical_crossentropy',metrics = ['accuracy'])

    return model

In [ ]:
crnn_parallel_param_grid = {'n_filters': [16,32,64,64,64],
                   'filter_size': (3,1),
                   'pool_size': [(2,2),(2,2),(2,2),(4,4),(4,4),(4,2)],
                   'dropout_rate': 0.3,
                   'n_cov_layers': 5, 
                   'n_recurrent_layers': 2,
                   'n_time_layers': 1,
                   'n_time_dst_nodes': 128,
                   'n_dense_nodes':240,
                   'l2': .001,
                   'output_shape': y_train.shape }

crnn_parallel = crnn_parallel(X_train, crnn_parallel_param_grid)
crnn_parallel_fitted , crnn_parallel_history = train_model(crnn_parallel, X_train, y_train, X_val, y_val,
                                                           model_name = 'crnn_parallel2',epochs = 50)

In [ ]:
train_loss_crnn_p, train_acc_crnn_p = crnn_parallel_fitted.evaluate(X_train, y_train)
print('Training set Accuracy: ',train_acc_crnn_p)

In [ ]:
val_loss_crnn_p, val_acc_crnn_p = crnn_parallel_fitted.evaluate(X_val, y_val)
print('Validation set Accuracy: ',val_acc_crnn_p)

In [ ]:
test_loss_crnn_p, test_acc_crnn_p = crnn_parallel_fitted.evaluate(X_test, y_test)
print('Test set Accuracy: ',test_acc_crnn_p)

In [ ]:
train_acc = [train_acc_cnn, train_acc_crnn, train_acc_crnn_p]
val_acc = [val_acc_cnn, val_acc_crnn, val_acc_crnn_p]
test_acc = [train_acc_cnn, test_acc_crnn,test_acc_crnn_p]
cols = ['CNN', 'CRNN', 'CRNN Parallel']
index = ['Training', 'Validation', 'Testing']
data = dict(zip(cols, [train_acc, val_acc, test_acc]))
results = pd.DataFrame(data, index = index)

In [ ]:
results

In [ ]:
train_predictions = crnn_parallel_fitted.predict(X_train)
val_predictions = crnn_parallel_fitted.predict(X_val)
test_predictions = crnn_parallel_fitted.predict(X_test)

In [ ]:
train_predictions_df = pd.DataFrame(train_predictions, columns = [l + '_prob' for l in enc.classes_], index = track_info.loc[track_info.split == 'training'].index )
val_predictions_df = pd.DataFrame(val_predictions, columns = [l + '_prob' for l in enc.classes_], index = track_info.loc[track_info.split == 'validation'].index )
test_predictions_df = pd.DataFrame(test_predictions, columns = [l + '_prob' for l in enc.classes_], index = track_info.loc[track_info.split == 'test'].index )

In [ ]:
train_predictions_df.to_pickle('train_predictions.pickle')
val_predictions_df.to_pickle('val_predictions.pickle')
test_predictions_df.to_pickle('test_predictions.pickle')